# Aufgabe 1 (5P)

## a)
Nutzen Sie vom Datensatz „pvtest.csv“ die Werte 'Edaily','Dci','Dcp','Dcu','Temp1','hour‘ und erstellen Sie einen DataFrame
als Zeitreihe, ähnlich wie unten abgebildet. Auf Grundlage dieser Daten soll der Energieertrag Dcp immer in 5 Minuten in
der Zukunft prognostiziert werden. (0.5P)

<img src='images/1a.png' width=400>

In [35]:
import pandas as pd

df = pd.read_csv('pvtest.csv')
df['Time'] = pd.to_datetime(df['Time'], format='%d.%m.%y %H:%M')
df['hour'] = df['Time'].dt.hour
df.head()

,Unnamed: 0,Time,Inverter,AcFreq,AcPower,Edaily,Eint,Etotal,Riso,Error,...,Street_plant,Latitude_plant,Longitude_plant,ModulArea_plant,SpecificEnergyExpected_plant,StartupDate_plant,TotalPower_plant,string_id,tags,hour
0,232,2021-01-20 07:45:00,WR 3 Diehl,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Lenbachstr. 17 - 27,49.988792,8.442296,417.18,900.0,31.10.12 00:00,61.74,600a4c362671ca676a1ece2855a46bb1,NaN,7
1,233,2021-01-20 07:50:00,WR 3 Diehl,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Lenbachstr. 17 - 27,49.988792,8.442296,417.18,900.0,31.10.12 00:00,61.74,600a4c362671ca676a1ece2855a46bb1,NaN,7
2,234,2021-01-20 07:55:00,WR 3 Diehl,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Lenbachstr. 17 - 27,49.988792,8.442296,417.18,900.0,31.10.12 00:00,61.74,600a4c362671ca676a1ece2855a46bb1,NaN,7
3,235,2021-01-20 08:00:00,WR 3 Diehl,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Lenbachstr. 17 - 27,49.988792,8.442296,417.18,900.0,31.10.12 00:00,61.74,600a4c362671ca676a1ece2855a46bb1,NaN,8
4,236,2021-01-20 08:05:00,WR 3 Diehl,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,Lenbachstr. 17 - 27,49.988792,8.442296,417.18,900.0,31.10.12 00:00,61.74,600a4c362671ca676a1ece2855a46bb1,NaN,8


## b)
Teilen Sie die Daten sinnvoll in Lern-und Testdatensatz auf, so dass die zeitlichen Reihenfolgen bestehen bleiben ! (0.5P)

In [36]:
from sklearn.model_selection import train_test_split

columns = ['Edaily','Dci','Dcu','Temp1','hour']
x = df[columns]
y = df['Dcp']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## c)
Standardisieren Sie die Daten mit dem Standardscaler und restrukturieren Sie die Zeitreihe in einzelne Datenfenster, so dass
Sie aus allen vorhandenen 6 Features aus den letzten drei Stunden immer jeweils den Energieertrag für die nächsten 5
Minuten vorhersagen können. (1,5P)

# d)
Definieren und kompilieren Sie jetzt ein passendes Modell in Keras, welches mind. eine LSTM-Layer nutzt und DropoutLayer einfügt, um „Überanpassung“ zu verhindern. (1P)

## e)
Trainieren Sie das Modell mit allen Lerndaten über mind. 100 Epochen, nutzen Sie ggf. auch callbacks, um den Lernprozess
zu verkürzen. Verbessern Sie Ihr Training und Ihre Modellarchitektur so lange, bis Sie einen Mean absolute Error von mind.
50 erzielen! (1P)

# f)
Visualisieren Sie Ihre Prognoseergebnisse, so ähnlich wie unten angegeben im Vergleich gemessener Daten mit
prognostizierten Werten! (0.5P)

<img src="images/1f.png" width=900>